In [43]:
# Mrityunjay kumar #
# CNN Convolution Networks #

In [44]:
import numpy as np
import tensorflow as tf
from __future__ import print_function
import input_data
tf.compat.v1.disable_eager_execution()

In [45]:
mnist = input_data.read_data_sets('MNIST/',one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [46]:
def add_layer(inputs,in_shape,out_shape,layer=None,activation_function=None,dropout=None):
    W = weight_variable(in_shape)
    b = bias_variable(out_shape)
    if layer is "conv":
        h_conv = activation_function(conv2d(inputs,W)+b)
        h_pool = max_pool_2x2(h_conv)
        return h_pool
    wx_plus_b = tf.compat.v1.matmul(inputs,W)+b
    output = activation_function(wx_plus_b)
    if dropout is None:
        return output
    output_dropout = tf.compat.v1.nn.dropout(output,keep_prob)
    return output_dropout

In [47]:
def compute_accuracy(x_v,y_v):
    global prediction
    ypred = sess.run(prediction,feed_dict={xs:x_v,keep_prob:1})
    accurate_counts = tf.compat.v1.equal(tf.compat.v1.argmax(y_v,1),tf.compat.v1.arg_max(ypred,1))
    accuracy = tf.compat.v1.reduce_mean(tf.compat.v1.cast(accurate_counts,dtype=tf.compat.v1.float32))
    results = sess.run(accuracy,feed_dict={xs:x_v,ys:y_v,keep_prob:1})
    return results

In [48]:
def weight_variable(shape):
    initial = tf.compat.v1.truncated_normal(shape, stddev=0.1)
    return tf.compat.v1.Variable(initial)

def bias_variable(shape):
    initial = tf.compat.v1.constant(0.1, shape=shape)
    return tf.compat.v1.Variable(initial)

In [49]:
def conv2d(x,W):
    # stride [1,x_movement,y_movement,1]
    # # Must have strides[0] = strides[3] = 1
    return tf.compat.v1.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [50]:
def max_pool_2x2(x):
    # stride [1,x_movement,y_movement,1]
    return tf.compat.v1.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [51]:
#input placeholders
xs = tf.compat.v1.placeholder(tf.compat.v1.float32,[None,784])
ys = tf.compat.v1.placeholder(tf.compat.v1.float32,[None,10])
keep_prob = tf.compat.v1.placeholder(tf.compat.v1.float32)
x_image = tf.compat.v1.reshape(xs,[-1,28,28,1])

In [52]:
##conv1 layer##
h_conv_pool_1 = add_layer(x_image,[5,5,1,32],[32],activation_function=tf.compat.v1.nn.relu,layer='conv')
# patch 5x5, in size 1, out size 32
# output size 28x28x32  (from conv2d)
# output size 14x14x32  (from max_pool_2x2)

In [53]:
##conv2 layer##
h_conv_pool_2 = add_layer(h_conv_pool_1,[5,5,32,64],[64],activation_function=tf.compat.v1.nn.relu,layer='conv')
# patch 5x5, in size 32, out size 64
# output size 14x14x64 (from conv2d)
# output size 7x7x64 (from conv2d)

In [54]:
##fc layer 1##
h_conv_pool_2_flat = tf.compat.v1.reshape(h_conv_pool_2,[-1,7*7*64])
fc_1 = add_layer(h_conv_pool_2_flat,[7*7*64,1024],[1024],activation_function=tf.compat.v1.nn.relu,dropout=True)

In [55]:
##fc layer 2##
prediction = add_layer(fc_1,[1024,10],[10],activation_function=tf.nn.softmax)

In [56]:
cross_entropy = tf.compat.v1.reduce_mean(-tf.compat.v1.reduce_sum(ys*tf.compat.v1.log(prediction),reduction_indices=[1]))

In [57]:
training = tf.compat.v1.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [58]:
init = tf.compat.v1.initialize_all_variables()

In [59]:
sess = tf.compat.v1.Session()

In [60]:
sess.run(init)

In [61]:
for i in range(1000):
    batch_x,label_y = mnist.train.next_batch(200)
    sess.run(training,feed_dict={xs:batch_x,ys:label_y,keep_prob:0.5})
    if i%50==0:
        print(compute_accuracy(mnist.test.images,mnist.test.labels))

W0128 16:22:42.273402 4579792320 deprecation.py:323] From <ipython-input-47-376053cd9e53>:4: arg_max (from tensorflow.python.ops.gen_math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.math.argmax` instead


0.1245
0.832
0.9069
0.9285
0.9408
0.9484
0.9548
0.9588
0.9631
0.9632
0.9652
0.9685
0.9687
0.9711
0.973
0.973
0.9758
0.9761
0.9756
0.9764
